In [197]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [91]:
#ser_ts_sum = train[train['trx_category'] == 'POS'][['cl_id', 'target_sum', 'year', 'month']].groupby(by=['cl_id', 'year', 'month']).sum().astype('int64')
#ser_pos_count = train[train['trx_category'] == 'POS'][['cl_id', 'target_sum', 'year', 'month']].groupby(by=['cl_id', 'year', 'month']).count().rename(columns={'target_sum':'count_POS'}).astype('int64')
#ser_notpos_count = train[train['trx_category'] != 'POS'][['cl_id', 'target_sum', 'year', 'month']].groupby(by=['cl_id', 'year', 'month']).count().rename(columns={'target_sum':'count_notPOS'}).astype('int64')
#train = train.drop(axis=1, labels='target_sum').join(ser_ts_sum, on=['cl_id', 'year', 'month'], how='left').join(ser_pos_count, on=['cl_id', 'year', 'month'], how='left').join(ser_notpos_count, on=['cl_id', 'year', 'month'], how='left')


'''
[description]
InsertDataFrame - pandas dataframe
ColumnName - name of transforming column
'''
def MakeDateYearMonth(InsertDataFrame, ColumnName):
    df = InsertDataFrame.copy()
    df[ColumnName] = pd.to_datetime(df[ColumnName])
    df['year'] = df[ColumnName].dt.year
    df['month'] = df[ColumnName].dt.month
    return df    

'''
[description]
InsertDataFrame - pandas dataframe
GroupByList - list of column names
TargetColumn - name of one column in dataframe, string
TargetName - name of returning column
'''
def groupbysum(InsertDataFrame, GroupByList, TargetColumn, TargetName):
    df = InsertDataFrame.copy()
    sdtype = df[TargetColumn].dtype
    ser = df.groupby(by=GroupByList).sum().rename(columns={TargetColumn:TargetName}).astype(sdtype)
    return ser
    
'''
[description]
InsertDataFrame - pandas dataframe
GroupByList - list of column names
TargetColumn - name of one column in dataframe, string
TargetName - name of returning column
'''
def groupbycount(InsertDataFrame, GroupByList, TargetColumn, TargetName):
    df = InsertDataFrame.copy()
    ser = df.groupby(by=GroupByList).count().rename(columns={TargetColumn:TargetName}).astype('int64')
    return ser    

'''
[description]
InsertDataFrame - pandas dataframe
SeriesList - list of pandas series
OnJoin - string or list of column name(s)
'''
def DfLeftJoinSer(InsertDataFrame, SeriesList, OnJoin):
    df = InsertDataFrame.copy()
    for i in SeriesList:
        df = df.join(i, on=OnJoin, how='left')
    return df

In [56]:
url_train = 'https://raw.githubusercontent.com/Hitriihoma/Machine-learning-course-task/master/train.csv'
url_test = 'https://raw.githubusercontent.com/Hitriihoma/Machine-learning-course-task/master/test.csv'

train = pd.read_csv(url_train)
test = pd.read_csv(url_test)

In [57]:
train2 = train.copy()

In [188]:
train = train2.copy()
train.head()

,PERIOD,cl_id,MCC,channel_type,currency,TRDATETIME,amount,trx_category,target_sum
0,01/10/2017,0,5200,NaN,810,21OCT17:00:00:00,5023.0,POS,0.0
1,01/10/2017,0,6011,NaN,810,12OCT17:12:24:07,20000.0,DEPOSIT,0.0
2,01/12/2017,0,5921,NaN,810,05DEC17:00:00:00,767.0,POS,0.0
3,01/10/2017,0,5411,NaN,810,21OCT17:00:00:00,2031.0,POS,0.0
4,01/10/2017,0,6012,NaN,810,24OCT17:13:14:24,36562.0,C2C_OUT,0.0


In [93]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 490513 entries, 0 to 490512
Data columns (total 9 columns):
PERIOD          490513 non-null object
cl_id           490513 non-null int64
MCC             490513 non-null int64
channel_type    487603 non-null object
currency        490513 non-null int64
TRDATETIME      490513 non-null object
amount          490513 non-null float64
trx_category    490513 non-null object
target_sum      490513 non-null float64
dtypes: float64(2), int64(3), object(4)
memory usage: 33.7+ MB


# Метрики для модели
год, месяц - сколько берём? надо смотреть сколько есть  
количество POS-транзакций  
количество не-POS-транзакций  
сумма POS-транзакций  
сумма не- POS-транзакций

In [189]:
# Входные сырые данные + колонки
train = MakeDateYearMonth(train, 'PERIOD')
group_list = ['cl_id', 'year', 'month']
ser_ts_sum = groupbysum(train[train['trx_category'] == 'POS'][['cl_id', 'target_sum', 'year', 'month']],
                       group_list,
                       'target_sum',
                       'target_sum')
ser_pos_amount_sum = groupbysum(train[train['trx_category'] == 'POS'][['cl_id', 'amount', 'year', 'month']],
                       group_list,
                       'amount',
                       'sum_pos')
ser_notpos_amount_sum = groupbysum(train[train['trx_category'] != 'POS'][['cl_id', 'amount', 'year', 'month']],
                       group_list,
                       'amount',
                       'sum_notpos')
ser_pos_count = groupbycount(train[train['trx_category'] == 'POS'][['cl_id', 'target_sum', 'year', 'month']],
                       group_list,
                       'target_sum',
                       'count_pos')
ser_notpos_count = groupbycount(train[train['trx_category'] != 'POS'][['cl_id', 'target_sum', 'year', 'month']],
                       group_list,
                       'target_sum',
                       'count_notpos')

train = DfLeftJoinSer(train.drop(axis=1, labels=['PERIOD','MCC', 'channel_type', 'currency', 'TRDATETIME', 'amount', 'trx_category', 'target_sum']).drop_duplicates(),
                      [ser_ts_sum, ser_pos_amount_sum, ser_notpos_amount_sum, ser_pos_count, ser_notpos_count],
                      ['cl_id', 'year', 'month'])
train.head()

,cl_id,year,month,target_sum,sum_pos,sum_notpos,count_pos,count_notpos
0,0,2017,1,0.00,7821.00,56562.00,3.0,2.0
5,1,2017,1,0.00,122578.04,145000.00,101.0,3.0
109,5,2017,1,36209504.25,290500.75,255868.38,111.0,31.0
251,9,2017,1,0.00,12365.09,836950.00,29.0,10.0
290,10,2017,1,0.00,200956.11,923387.88,374.0,89.0


In [198]:
# преобразуем в "последний месяц, предпоследний месяц, предпредпоследний месяц
train = train.sort_values(by=['cl_id', 'year', 'month'], ascending=[True, False, False])
train['1_sum_pos'] = np.NaN
for i in train.cl_id[0:6]:
    mon = 1
    for ji in train[train['cl_id'] == i].index:
        up_index = train[train['cl_id'] == i].index[0]
        train.iloc[up_index][f'{mon}_sum_pos'] = train.iloc[ji]['sum_pos']
        train.iloc[up_index][f'{mon}_sum_notpos'] = train.iloc[ji]['sum_notpos']
        train.iloc[up_index][f'{mon}_count_pos'] = train.iloc[ji]['count_pos']
        train.iloc[up_index][f'{mon}_count_notpos'] = train.iloc[ji]['count_notpos']
        mon+=1
#train.drop(axis=1, labels=['year', 'month, 'sum_pos', 'sum_notpos', 'count_pos', ['count_notpos']).drop_duplicates() 
train

C:\Games\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Games\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\DNS\AppData\Roaming\Python\Python37\site-packages\pandas\core\series.py:1015: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.loc[key] = value
C:\Games\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A v

,cl_id,year,month,target_sum,sum_pos,sum_notpos,count_pos,count_notpos,1_sum_pos
0,0,2017,1,0.00,7821.00,56562.00,3.0,2.0,NaN
5,1,2017,1,0.00,122578.04,145000.00,101.0,3.0,NaN
109,5,2017,1,36209504.25,290500.75,255868.38,111.0,31.0,NaN
251,9,2017,1,0.00,12365.09,836950.00,29.0,10.0,NaN
290,10,2017,1,0.00,200956.11,923387.88,374.0,89.0,NaN
753,11,2017,1,0.00,245747.14,182000.00,207.0,10.0,NaN
970,14,2017,1,24786532.32,258990.17,189700.00,132.0,4.0,NaN
1106,20,2017,1,0.00,222308.09,215000.00,74.0,3.0,NaN
1188,21,2017,1,0.00,75433.06,47778.00,35.0,5.0,NaN
1183,21,2016,1,0.00,144670.59,71000.00,78.0,6.0,NaN


# Анализ данных
какой месяц для модели текущий  
сколько для каждого клиента известно месяцев

In [184]:
# количество уникальных клиентов
print('number of cl_id', len(train.cl_id.unique()), end='\n\n')
print('count of months \n', train.groupby('cl_id').count().year.value_counts(), end='\n\n')
temp_df = train.groupby('cl_id').last()[['year', 'month', 'target_sum']]
temp_df['ym'] = temp_df.apply(lambda row: str(int(row.year))+'-'+str(int(row.month)), axis=1)
print('last month of target_sum > 1 \n', temp_df[temp_df['target_sum'] > 1].ym.value_counts(), end='\n\n' )
print('last month of target_sum < 1 \n', temp_df[temp_df['target_sum'] < 1].ym.value_counts(), end='\n\n' )

number of cl_id 5000

count of months 
 1    3448
2    1550
3       2
Name: year, dtype: int64

count of target_sum > 1 
 2017-1    2261
2016-1     420
2018-1      86
Name: ym, dtype: int64

count of target_sum < 1 
 2017-1    1715
2016-1     468
2018-1      50
Name: ym, dtype: int64



In [176]:
train

,cl_id,year,month,target_sum,pos_sum,notpos_sum,count_pos,count_notpos
0,0,2017,1,0.00,7821.00,56562.00,3.0,2.0
5,1,2017,1,0.00,122578.04,145000.00,101.0,3.0
109,5,2017,1,36209504.25,290500.75,255868.38,111.0,31.0
251,9,2017,1,0.00,12365.09,836950.00,29.0,10.0
290,10,2017,1,0.00,200956.11,923387.88,374.0,89.0
753,11,2017,1,0.00,245747.14,182000.00,207.0,10.0
970,14,2017,1,24786532.32,258990.17,189700.00,132.0,4.0
1106,20,2017,1,0.00,222308.09,215000.00,74.0,3.0
1183,21,2016,1,0.00,144670.59,71000.00,78.0,6.0
1184,22,2017,1,0.00,110267.00,139500.00,45.0,14.0


In [191]:
'''for i in train.cl_id[10:30]:
    fig = plt.figure()
    x = list()
    y = list()
    temp_df = train[train['cl_id'] == i].reset_index(drop=True)
    for ji in temp_df.index:
        j = temp_df.iloc[ji]
        x.append(str(int(j.year))+ '-' + str(int(j.month)))
        y.append(j.pos_sum + j.notpos_sum)
    x.append('target')
    y.append(j.target_sum)
    print(x,y)
    plt.plot(x,y)
    plt.show()
plt.close('all')  
temp_df'''

"for i in train.cl_id[10:30]:\n    fig = plt.figure()\n    x = list()\n    y = list()\n    temp_df = train[train['cl_id'] == i].reset_index(drop=True)\n    for ji in temp_df.index:\n        j = temp_df.iloc[ji]\n        x.append(str(int(j.year))+ '-' + str(int(j.month)))\n        y.append(j.pos_sum + j.notpos_sum)\n    x.append('target')\n    y.append(j.target_sum)\n    print(x,y)\n    plt.plot(x,y)\n    plt.show()\nplt.close('all')  \ntemp_df"

# Формирование выборки

In [63]:
# Пример формирования тестовой выборки. Возьмем 10 случайных пользвоателей.
users_sample = train['cl_id'].sample(10).values
user_sample_df = train[(train['cl_id'].isin(users_sample))][:]
user_sample_df.head(5)

,PERIOD,cl_id,MCC,channel_type,currency,TRDATETIME,amount,trx_category,year,month,target_sum,count_POS,count_notPOS
48794,2017-01-04,901,5814,type2,810,29APR17:00:00:00,402.0,POS,2017,1,58147131.81,223.0,17.0
48795,2017-01-04,901,5261,type2,810,15APR17:00:00:00,28490.5,POS,2017,1,58147131.81,223.0,17.0
48799,2017-01-03,901,5921,type2,810,03MAR17:00:00:00,389.0,POS,2017,1,58147131.81,223.0,17.0
48803,2017-01-04,901,5814,type2,810,14APR17:00:00:00,99.0,POS,2017,1,58147131.81,223.0,17.0
48804,2017-01-04,901,5921,type2,810,20APR17:00:00:00,385.0,POS,2017,1,58147131.81,223.0,17.0


In [64]:
# Для данных пользователей необходимо спрогнозировать рассчитанную величину'target_sum'.
user_sample_df[['cl_id','target_sum']].drop_duplicates()

,cl_id,target_sum
48794,901,58147131.81
74524,1339,7079905.92
74617,1339,2876211.78
99591,1813,921600.00
134218,2414,2637756.90
152544,2715,48508409.67
261930,4977,0.00
352786,7082,0.00
359055,7213,0.00
404364,8274,151400.62
